In [1]:
import subprocess
import pandas as pd
import numpy as np

In [2]:

complete_fasta=""
filepath="progenomes_rnarobo_res.txt"
filepath_fasta="progenomes3.contigs.representatives.fasta"
f=open(filepath)
f_l=f.readlines()
#going thorugh the results in one file: check for start of found seq ("  "), then append the start position, end position, sequencename (important!) and sequence itself
counter=0
#start=[]
#end=[]
seqnam=[]
sequence=[]
#go trhough each line of rnarobo output
for line in f_l[11:-6]:
    #if len(line)==0:
       # break
    if not line[0]=="|":
        line=line.strip().split()
        #start.append(line[0])
        #end.append(line[1])
        seqnam.append(line[2])
        counter=1
    #add the sequence of the ribozyme    
    elif counter==1:
        line=line.strip()
        sequence.append(line)
        counter=0

#all reads where a ribozyme was found        
unique_seqnam=set(seqnam)
#look for all the 
for seqname in unique_seqnam:
    inp= "sed -n '/"+seqname+"/,/>/p' "+filepath_fasta
    l=subprocess.run(inp,shell=True,capture_output=True,text=True)
    complete_fasta+=l.stdout
    #print(l.stdout)
com_fas=complete_fasta.split("\n")
fasta_corr=[]
checker=0
fasta_part=""
#to -1 because last one is empty 
for line in com_fas:

    if line=="": 
        continue

    elif line[0]==">":


        if checker==0:
            fasta_corr.append(fasta_part)
            fasta_part=""
            line=line.split()
            fasta_corr.append(line[0])
            checker=1
        else:
            checker=0

    else:
        fasta_part+=line.strip()


fasta_corr=fasta_corr[1:]

fast_seq="\n".join(fasta_corr)

fast_seq=fast_seq.replace(".","_")


f= open("combined_all_viral_sequences_v2.fasta","w+")
f.write(fast_seq)
f.close()    


inp2="/mnt/mnemo6/lukas/anaconda3/envs/sar3/bin/tRNAscan-SE -G combined_all_viral_sequences_v2.fasta -o combined_trna_scan_v2.txt"
subprocess.run(inp2, shell=True)

CompletedProcess(args='/mnt/mnemo6/lukas/anaconda3/envs/sar3/bin/tRNAscan-SE -G combined_all_viral_sequences_v2.fasta -o combined_trna_scan_v2.txt', returncode=0)

In [3]:
rnarobo_=open("progenomes_rnarobo_res.txt")
rnarobo=rnarobo_.readlines()
rnarobo_.close()

In [4]:
trna_=open("combined_trna_scan_v2.txt")
trna=trna_.readlines()
trna_.close()

In [66]:
#rnarobo[11:-6] gives us the part we need

In [74]:
#trna[3:] gives us the part we need

In [5]:
dicti={}
for line in trna[3:]:
    #create a dictionary with seq name as key and the starting position as value
        line=line.strip().split()
        if line[0] in dicti:
            dicti[line[0]].append(int(line[3]))
        else:
            dicti[line[0]]=[int(line[3])]
            
        
        

In [6]:
dicti_r={}
for line in rnarobo[11:-6]:
    if line[0]!="|":
        line=line.replace(".","_")
        line=line.strip().split()
        if line[2] in dicti_r:
            dicti_r[line[2]].append(int(line[0]))
        else:
            dicti_r[line[2]]=[int(line[0])]
        

In [15]:
matching=[]
for key in dicti_r:
    if key in dicti:
        for value in dicti_r[key]:
            for value2 in dicti[key]:
                if value-value2 in range(-4,4):
                    matching.append([key,value,value2])

In [16]:
#make sure everything is according to plan and we have no duplicates etc.
newm=[]
for mat in matching:
    if mat not in newm:
        newm.append(mat)

In [17]:
lets_do_this=pd.DataFrame(columns=["ID","trna_nr","trna_start","trna_end","trna_type","codon","ins_start","ins_end","trna_confidence","psuedo","rnarobo_start","rnarobo_end","rnarobo_seq"])
check=0
duplicates=[]
for nr in range(len(newm)):
    match=newm[nr]
    lin_df=""
    for line in trna[3:]:
    #create a dictionary with seq name as key and the starting position as value
        line=line.strip().split()
        
        if match[0]==line[0]:
            if len(lin_df.split())==0:
                if str(match[2])==line[3]:
                    #print(line)
                    lin_df+=" ".join(line)+" "
                    if lin_df[-2]!="o":
                        lin_df+="NA "
            else: 
                duplicates.append(line)
    for line in rnarobo[11:-6]:
        line=line.replace(".","_")
        line=line.strip().split()
        if len(lin_df.split())<13:

            if check==1:
                #if len(lin_df.split())<13:
                    lin_df+=str(line[0])
                    check=0
                #else:
                        #duplicates.append(line)

                #check=0
             #####FIX 2: Important here: we did this to make sure that we are actually matching the right ribozyme here...
            if len(line)>1:
                #here we made the mistake: not checking the actual ID but only starting position
                if match[0]==line[2]:
                    if str(match[1])==line[0]:
                    #if len(lin_df.split())<13:
                        lin_df+=" ".join(line[:2])
                        lin_df+=" "
                        check=1
            #else:
                #duplicates.append(line)
                #check=0
    lin_f=lin_df.split()
    lets_do_this.loc[nr]=np.array(lin_f)
    #lets_do_this.iloc[nr,:8]=lin_df[0]
    #lets_do_this.iloc[nr,8:-1]=lin_df[1]
    #lets_do_this.iloc[nr,-1]=lin_df[2]

In [18]:
lets_do_this

,ID,trna_nr,trna_start,trna_end,trna_type,codon,ins_start,ins_end,trna_confidence,psuedo,rnarobo_start,rnarobo_end,rnarobo_seq
